In [1]:
from  geopy.geocoders import Nominatim
from aircraft_env import AircraftEnv
import xarray as xr
%load_ext autoreload
%autoreload 2
geolocator = Nominatim(user_agent='myapplication')
# city ="Agra"
# country ="India"
loc_1 = geolocator.geocode('Montréal–Trudeau International Airport')
print("latitude is :-" ,loc_1.latitude,"\nlongtitude is:-" ,loc_1.longitude)

loc_2 = geolocator.geocode('Charles de Gaulle Airport')
print("latitude is :-" ,loc_2.latitude,"\nlongtitude is:-" ,loc_2.longitude)

env = AircraftEnv(arrival_location=loc_1, destination=loc_2, n_waypoints=9, deviation=-10)

latitude is :- 45.46873715 
longtitude is:- -73.74257166095532
latitude is :- 49.0068908 
longtitude is:- 2.5710819691019156


In [9]:
from scipy import interpolate
import re
import os
import time

wps = env._waypoints_to_array()
def get_wind_speed(ds, key=""):
    assert key in ['WIND_P0_L100_GLL0', 'WDIR_P0_L100_GLL0']
    lat = ds['lat_0'].data
    lon = ds['lon_0'].data - 360
    wind = ds[key].data.T
    f = interpolate.interp2d(lat, lon, wind, kind='cubic')
    return f


lst = list(filter(lambda x: x.endswith("grib2"), os.listdir("wind_info")))
re_exp_wind = re.compile(r'ISBL_\d+_latlon')
dct = {"wdir": [], "wdir_pressures": [], "wind": [], "wind_pressures": []}

def add_to_dict(mod, file, dct):
    if mod in file:
        wdirs = []
        dct[f'{mod.lower()}_pressures'].append(pressure)
        f = get_wind_speed(ds, key=f'{mod}_P0_L100_GLL0')
        for trajectory in wps:
            wdir_wp = f(trajectory[:,0], trajectory[:,1]).diagonal().tolist()
            wdirs.append(wdir_wp)
        dct[f'{mod.lower()}'].append(wdirs)

start = time.time()
for file in lst:
    found_pattern = re_exp_wind.search(file)
    if found_pattern is not None:
        pressure = int(found_pattern.group(0).split('_')[1])
        ds = xr.open_dataset(os.path.join("wind_info", file), engine="pynio")
        add_to_dict("WDIR", file, dct)
        add_to_dict("WIND", file, dct)
print(time.time() - start)
#         if "WIND" in file:
#             dct['wind_pressures'].append(pressure)
#             f = get_wind_speed(ds, key='WIND_P0_L100_GLL0')
#             wind_wp = f(wps[:,0], wps[:,1]).diagonal().tolist()
#             dct['wind'].append(wind_wp)
            
# file = lst[0]
# ds = xr.open_dataset(os.path.join("wind_info", file), engine="pynio")


66.4720766544342


In [11]:
result1, result2 = zip(*sorted(zip(dct['wdir_pressures'], dct['wdir']), key=lambda x: x[0]))
dct['wdir_pressures'] = list(result1)
dct['wdir'] = list(result2)
# result1, result2

In [12]:
result1, result2 = zip(*sorted(zip(dct['wind_pressures'], dct['wind']), key=lambda x: x[0]))
dct['wind_pressures'] = list(result1)
dct['wind'] = list(result2)

In [14]:
import json
# for i in range(len(dct['wdirs'])):
#     dct['wdirs'][i] = dct['wdirs'][i].tolist()
# for i in range(len(dct['wind'])):
#     dct['wind'][i] = dct['wind'][i].tolist()

#     dct['wdirs']
#     dct[key]['wdir']= dct[key]['wdir'].tolist() 
#     dct[key]['wind']= dct[key]['wind'].tolist() 
with open("wind.json", 'w') as f:
    json.dump(dct, f,  indent=4)